In [1]:
!pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
!pip install matplotlib
%cd /root/text_to_mesh

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-wddw5nk8
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-wddw5nk8
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit c00e7c6bd40f1cd56c369f598ad357c591cd12e1
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
/root/text_to_mesh


In [2]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
        codebook_size =  2048, 
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

In [10]:

torch.cuda.empty_cache()
gc.collect()  
 
transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,  
    fine_pre_gateloop_depth= 2, 
    attn_depth = 12,  
    attn_heads = 12, 
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    cross_attn_num_mem_kv = 4,
    num_sos_tokens = 1, 
    dropout  = 0.0,
    max_seq_len = 1500, 
    fine_attn_depth = 2,
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", 
    text_condition_cond_drop_prob = 0.25, 
).to("cuda")
pkg = torch.load("./MeshGPT-transformer_trained_01.pt") 
transformer.load_state_dict(pkg['model'],strict=False)
 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<All keys matched successfully>

In [11]:
dataset = MeshDataset.load("./labels_885_10x5_21720_mod.npz") 
labels = list(set(item['texts'] for item in dataset.data))
dataset.generate_codes(autoencoder,150)
dataset.data[0].keys() 
dataset.embed_texts(transformer,1)
 

[MeshDataset] Loaded 15170 entrys
[MeshDataset] Created from 15170 entrys


100%|██████████| 102/102 [00:06<00:00, 15.99it/s]


[MeshDataset] Generated codes for 15170 entrys
[MeshDataset] Generated 885 text_embeddings


In [12]:
 
torch.cuda.empty_cache()
gc.collect()  
 
batch_size =16
grad_accum_every =4      
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                 learning_rate = 1e-4, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(740, stop_at_loss = 0.00005)   

Epoch 1/740:   0%|          | 1/948 [00:00<03:04,  5.12it/s, loss=0.49]

Epoch 1/740:   6%|▌         | 58/948 [00:13<03:33,  4.16it/s, loss=1.05]  


OutOfMemoryError: CUDA out of memory. Tried to allocate 254.00 MiB. GPU 

In [ ]:
pkg = dict( model = transformer.state_dict(), ) 
torch.save(pkg, str("./MeshGPT-transformer_trained.pt"))

In [ ]:
from meshgpt_pytorch import mesh_render
text_coords = []
rows = []
transformer.eval() 

for text in ["box", "hose" ,"rake" ,"shovel", "cart", "bin","trash can" ,"ladder" ,"chair", "table",  "bed"] :
    print("Generating ", text) 
    face_coords = transformer.generate(texts = [text] ,  temperature = 0.0)
    text_coords.append(face_coords)

rows.append(text_coords)
mesh_render.combind_mesh_with_rows(f'./rows.obj', rows)

Generating  box


  2%|▏         | 31/1500 [00:00<00:04, 295.15it/s]

 21%|██        | 312/1500 [00:01<00:03, 311.30it/s]


Generating  hose


 67%|██████▋   | 1002/1500 [00:03<00:01, 313.93it/s]


...

 43%|████▎     | 648/1500 [00:02<00:02, 313.30it/s]


Generating  table


 58%|█████▊    | 864/1500 [00:02<00:02, 312.12it/s]


Generating  bed


 85%|████████▍ | 1272/1500 [00:04<00:00, 313.31it/s]
